In [ ]:
%pip install delta-spark

Python interpreter will be restarted.
Python interpreter will be restarted.


In [ ]:
import pandas as pd
import re
import logging
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

# Inicializar SparkSession
spark = SparkSession.builder.appName("dataUnified").getOrCreate()

In [ ]:
# Configuração do Logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# Criar um logger
logger = logging.getLogger(__name__)

logger.info("Criando sessão Spark....")

In [ ]:
schema = StructType([
    StructField("ano", StringType()),
    StructField("trimestre", StringType()),
    StructField("categoria", StringType()),
    StructField("tipo", StringType()),
    StructField("cnpj_if", StringType()),
    StructField("instituicao_financeira", StringType()),
    StructField("indice", StringType()),
    StructField("quantidade_de_reclamacoes_reguladas_procedentes", StringType()),
    StructField("quantidade_de_reclamacoes_reguladas_outras", StringType()),
    StructField("quantidade_de_reclamacoes_nao_reguladas", StringType()),
    StructField("quantidade_total_de_reclamacoes", StringType()),
    StructField("quantidade_total_de_clientes_ccs_e_scr", StringType()),
    StructField("quantidade_de_cliente_ccs", StringType()),
    StructField("quantidade_de_cliente_scr", StringType()),
    StructField("employer_website", StringType()),
    StructField("employer_headquarters", StringType()),
    StructField("employer_founded", StringType()),
    StructField("employer_industry", StringType()),
    StructField("employer_revenue", StringType()),
    StructField("geral", StringType()),
    StructField("cultura_e_valores", StringType()),
    StructField("diversidade_e_inclusao", StringType()),
    StructField("qualidade_de_vida", StringType()),
    StructField("alta_lideranca", StringType()),
    StructField("remuneracao_e_beneficios", StringType()),
    StructField("oportunidades_de_carreira", StringType()),
    StructField("recomendam_para_outras_pessoas", StringType()),
    StructField("perspectiva_positiva_da_empresa", StringType()),
    StructField("cnpj", StringType()),
    StructField("nome", StringType()),
    StructField("segmento", StringType())
])

logger.info("Criando Delta Table Schema")
try:
    # Criar um DataFrame vazio com o schema definido
    df_schema = spark.createDataFrame([], schema=schema)
    # Mostrar o DataFrame e o schema
    df_schema.printSchema()
    logger.info("Delta Table Schema - Criado com sucesso!")

except Exception as e:
    logger.error(f"Ocorreu um erro ao criar Delta Table Schema: {e}")

root
 |-- ano: string (nullable = true)
 |-- trimestre: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- cnpj_if: string (nullable = true)
 |-- instituicao_financeira: string (nullable = true)
 |-- indice: string (nullable = true)
 |-- quantidade_de_reclamacoes_reguladas_procedentes: string (nullable = true)
 |-- quantidade_de_reclamacoes_reguladas_outras: string (nullable = true)
 |-- quantidade_de_reclamacoes_nao_reguladas: string (nullable = true)
 |-- quantidade_total_de_reclamacoes: string (nullable = true)
 |-- quantidade_total_de_clientes_ccs_e_scr: string (nullable = true)
 |-- quantidade_de_cliente_ccs: string (nullable = true)
 |-- quantidade_de_cliente_scr: string (nullable = true)
 |-- employer_website: string (nullable = true)
 |-- employer_headquarters: string (nullable = true)
 |-- employer_founded: string (nullable = true)
 |-- employer_industry: string (nullable = true)
 |-- employer_revenue: string (nullable = 

In [ ]:
logger.info("Carregando parquet da camada Gold")
try:
    df_dadosunificados = spark.read.format("parquet") \
        .option("header", "true") \
        .schema(schema) \
        .load("/FileStore/glassdoor/gold/dadosunificados.parquet")
    
    logger.info("Parquet carregado com sucesso")

except Exception as e:
    logger.error(f"Ocorreu um erro ao criar Delta Table Schema: {e}")

ano,trimestre,categoria,tipo,cnpj_if,instituicao_financeira,indice,quantidade_de_reclamacoes_reguladas_procedentes,quantidade_de_reclamacoes_reguladas_outras,quantidade_de_reclamacoes_nao_reguladas,quantidade_total_de_reclamacoes,quantidade_total_de_clientes_ccs_e_scr,quantidade_de_cliente_ccs,quantidade_de_cliente_scr,employer_website,employer_headquarters,employer_founded,employer_industry,employer_revenue,geral,cultura_e_valores,diversidade_e_inclusao,qualidade_de_vida,alta_lideranca,remuneracao_e_beneficios,oportunidades_de_carreira,recomendam_para_outras_pessoas,perspectiva_positiva_da_empresa,cnpj,nome,segmento
2021,1o,GRUPO SECUNDARIO,BANCO/FINANCEIRA,62331228,DEUTSCHE BANK S.A. - BANCO ALEMAO,,0,null,0,1,2112,null,null,null,null,null,null,null,3.9,3.8,4.0,3.8,3.5,3.6,3.7,null,null,62331228,DEUTSCHE,S3
2021,2o,DEMAIS BANCOS E FINANCEIRAS,BANCO/FINANCEIRA,60518222,BANCO SUMITOMO MITSUI BRASILEIRO S.A.,,0,null,1,1,1063,null,null,null,null,null,null,null,3.4,3.2,3.3,3.4,3.1,3.3,3.1,null,null,60518222,BANCO SUMITOMO MITSUI BRASIL S.A.,null
2022,4o,"DEMAIS BANCOS, FINANCEIRAS E INSTITUICOES DE PAGAMENTO",BANCO/FINANCEIRA,60518222,BANCO SUMITOMO MITSUI BRASILEIRO S.A.,,0,null,1,1,1180,null,null,null,null,null,null,null,3.4,3.2,3.3,3.4,3.1,3.3,3.1,null,null,60518222,BANCO SUMITOMO MITSUI BRASIL S.A.,null
2021,1o,GRUPO SECUNDARIO,BANCO/FINANCEIRA,58616418,BANCO FIBRA S.A.,,1,null,2,5,163619,null,null,null,null,null,null,null,4.3,4.3,4.3,3.8,3.7,4.2,3.6,null,null,58616418,BANCO FIBRA S.A.,S3
2021,2o,DEMAIS BANCOS E FINANCEIRAS,BANCO/FINANCEIRA,58616418,BANCO FIBRA S.A.,,0,null,3,5,163669,null,null,null,null,null,null,null,4.3,4.3,4.3,3.8,3.7,4.2,3.6,null,null,58616418,BANCO FIBRA S.A.,S3
2021,3o,DEMAIS BANCOS E FINANCEIRAS,BANCO/FINANCEIRA,58616418,BANCO FIBRA S.A.,,0,null,1,1,163670,null,null,null,null,null,null,null,4.3,4.3,4.3,3.8,3.7,4.2,3.6,null,null,58616418,BANCO FIBRA S.A.,S3
2021,4o,"DEMAIS BANCOS, FINANCEIRAS E INSTITUICOES DE PAGAMENTO",BANCO/FINANCEIRA,58616418,BANCO FIBRA S.A.,,1,null,0,3,164195,null,null,null,null,null,null,null,4.3,4.3,4.3,3.8,3.7,4.2,3.6,null,null,58616418,BANCO FIBRA S.A.,S3
2022,1o,"DEMAIS BANCOS, FINANCEIRAS E INSTITUICOES DE PAGAMENTO",BANCO/FINANCEIRA,58616418,BANCO FIBRA S.A.,,1,null,0,2,163906,null,null,null,null,null,null,null,4.3,4.3,4.3,3.8,3.7,4.2,3.6,null,null,58616418,BANCO FIBRA S.A.,S3
2022,3o,"DEMAIS BANCOS, FINANCEIRAS E INSTITUICOES DE PAGAMENTO",BANCO/FINANCEIRA,58616418,BANCO FIBRA S.A.,,0,null,0,1,163544,null,null,null,null,null,null,null,4.3,4.3,4.3,3.8,3.7,4.2,3.6,null,null,58616418,BANCO FIBRA S.A.,S3
2021,2o,DEMAIS BANCOS E FINANCEIRAS,BANCO/FINANCEIRA,60518222,BANCO SUMITOMO MITSUI BRASILEIRO S.A.,,0,null,1,1,1063,null,null,null,null,null,null,null,3.4,3.3,3.2,3.4,3.1,3.3,3.1,null,null,60518222,BANCO SUMITOMO MITSUI BRASIL S.A.,S3


In [ ]:
logger.info("Refinamento e limpeza nas colunas")
try:
    # Função para limpar nomes de colunas
    def clean_column_name(name):
        # Substituir espaços e caracteres especiais por underscore
        cleaned_name = re.sub(r'[ ,;{}()\n\t=]', '_', name)
        cleaned_name = re.sub(r'[\[\]]', '', cleaned_name)  # Remover colchetes se presentes
        return cleaned_name

    # Aplicar a limpeza no schema existente
    new_fields = [StructField(clean_column_name(f.name), f.dataType, f.nullable) for f in schema.fields]
    cleaned_schema = StructType(new_fields)
    logger.info("Refinamento e limpeza realizado com sucesso!")

except Exception as e:
    logger.error(f"Ocorreu um erro ao refinar colunas: {e}")

root
 |-- ano: string (nullable = true)
 |-- trimestre: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- cnpj_if: string (nullable = true)
 |-- instituicao_financeira: string (nullable = true)
 |-- indice: string (nullable = true)
 |-- quantidade_de_reclamacoes_reguladas_procedentes: string (nullable = true)
 |-- quantidade_de_reclamacoes_reguladas_outras: string (nullable = true)
 |-- quantidade_de_reclamacoes_nao_reguladas: string (nullable = true)
 |-- quantidade_total_de_reclamacoes: string (nullable = true)
 |-- quantidade_total_de_clientes_ccs_e_scr: string (nullable = true)
 |-- quantidade_de_cliente_ccs: string (nullable = true)
 |-- quantidade_de_cliente_scr: string (nullable = true)
 |-- employer_website: string (nullable = true)
 |-- employer_headquarters: string (nullable = true)
 |-- employer_founded: string (nullable = true)
 |-- employer_industry: string (nullable = true)
 |-- employer_revenue: string (nullable = 

ano,trimestre,categoria,tipo,cnpj_if,instituicao_financeira,indice,quantidade_de_reclamacoes_reguladas_procedentes,quantidade_de_reclamacoes_reguladas_outras,quantidade_de_reclamacoes_nao_reguladas,quantidade_total_de_reclamacoes,quantidade_total_de_clientes_ccs_e_scr,quantidade_de_cliente_ccs,quantidade_de_cliente_scr,employer_website,employer_headquarters,employer_founded,employer_industry,employer_revenue,geral,cultura_e_valores,diversidade_e_inclusao,qualidade_de_vida,alta_lideranca,remuneracao_e_beneficios,oportunidades_de_carreira,recomendam_para_outras_pessoas,perspectiva_positiva_da_empresa,cnpj,nome,segmento
2021,1o,GRUPO SECUNDARIO,BANCO/FINANCEIRA,62331228,DEUTSCHE BANK S.A. - BANCO ALEMAO,,0,null,0,1,2112,null,null,null,null,null,null,null,3.9,3.8,4.0,3.8,3.5,3.6,3.7,null,null,62331228,DEUTSCHE,S3
2021,2o,DEMAIS BANCOS E FINANCEIRAS,BANCO/FINANCEIRA,60518222,BANCO SUMITOMO MITSUI BRASILEIRO S.A.,,0,null,1,1,1063,null,null,null,null,null,null,null,3.4,3.2,3.3,3.4,3.1,3.3,3.1,null,null,60518222,BANCO SUMITOMO MITSUI BRASIL S.A.,null
2022,4o,"DEMAIS BANCOS, FINANCEIRAS E INSTITUICOES DE PAGAMENTO",BANCO/FINANCEIRA,60518222,BANCO SUMITOMO MITSUI BRASILEIRO S.A.,,0,null,1,1,1180,null,null,null,null,null,null,null,3.4,3.2,3.3,3.4,3.1,3.3,3.1,null,null,60518222,BANCO SUMITOMO MITSUI BRASIL S.A.,null
2021,1o,GRUPO SECUNDARIO,BANCO/FINANCEIRA,58616418,BANCO FIBRA S.A.,,1,null,2,5,163619,null,null,null,null,null,null,null,4.3,4.3,4.3,3.8,3.7,4.2,3.6,null,null,58616418,BANCO FIBRA S.A.,S3
2021,2o,DEMAIS BANCOS E FINANCEIRAS,BANCO/FINANCEIRA,58616418,BANCO FIBRA S.A.,,0,null,3,5,163669,null,null,null,null,null,null,null,4.3,4.3,4.3,3.8,3.7,4.2,3.6,null,null,58616418,BANCO FIBRA S.A.,S3
2021,3o,DEMAIS BANCOS E FINANCEIRAS,BANCO/FINANCEIRA,58616418,BANCO FIBRA S.A.,,0,null,1,1,163670,null,null,null,null,null,null,null,4.3,4.3,4.3,3.8,3.7,4.2,3.6,null,null,58616418,BANCO FIBRA S.A.,S3
2021,4o,"DEMAIS BANCOS, FINANCEIRAS E INSTITUICOES DE PAGAMENTO",BANCO/FINANCEIRA,58616418,BANCO FIBRA S.A.,,1,null,0,3,164195,null,null,null,null,null,null,null,4.3,4.3,4.3,3.8,3.7,4.2,3.6,null,null,58616418,BANCO FIBRA S.A.,S3
2022,1o,"DEMAIS BANCOS, FINANCEIRAS E INSTITUICOES DE PAGAMENTO",BANCO/FINANCEIRA,58616418,BANCO FIBRA S.A.,,1,null,0,2,163906,null,null,null,null,null,null,null,4.3,4.3,4.3,3.8,3.7,4.2,3.6,null,null,58616418,BANCO FIBRA S.A.,S3
2022,3o,"DEMAIS BANCOS, FINANCEIRAS E INSTITUICOES DE PAGAMENTO",BANCO/FINANCEIRA,58616418,BANCO FIBRA S.A.,,0,null,0,1,163544,null,null,null,null,null,null,null,4.3,4.3,4.3,3.8,3.7,4.2,3.6,null,null,58616418,BANCO FIBRA S.A.,S3
2021,2o,DEMAIS BANCOS E FINANCEIRAS,BANCO/FINANCEIRA,60518222,BANCO SUMITOMO MITSUI BRASILEIRO S.A.,,0,null,1,1,1063,null,null,null,null,null,null,null,3.4,3.3,3.2,3.4,3.1,3.3,3.1,null,null,60518222,BANCO SUMITOMO MITSUI BRASIL S.A.,S3


In [ ]:
logger.info(f"DELTATABLE: {df_dadosunificados}'")
try:
    if spark._jsparkSession.catalog().tableExists("glassdoor_tb"):
        # # Escrever o DataFrame no Delta Table
        logger.info("Gravando o DataFrame no Delta Table: 'glassdoor_tb'")
        logger.info("Criando Partition Key = 'ano'....")
        df_dadosunificados.write.format("delta") \
            .option("overwriteSchema", "true") \
            .partitionBy("ano") \
            .mode("overwrite") \
            .save("/FileStore/default/tables/glassdoor_tb")
    
    logger.info("o DataFrame no Delta Table: 'glassdoor_tb realizado com sucesso!")
    logger.info("Salvando DataFrame 'glassdoor_tb' estruturado na camada GOLD")
    df_dadosunificados.write.mode("overwrite").partitionBy("ano").parquet("/FileStore/glassdoor/gold/glassdoor_tb")
    # Exibir o schema do DataFrame resultante
    logger.info(df_dadosunificados.printSchema())
    # Imprimir o DataFrame
    display(df_dadosunificados)
    logger.info("O DataFrame 'glassdoor_tb' salvo com sucesso!")

except Exception as e:
    logger.error(f"Ocorreu um erro: {e}")

In [ ]:
%sql
select * from delta.`/FileStore/default/tables/glassdoor_tb`;

ano,trimestre,categoria,tipo,cnpj_if,instituicao_financeira,indice,quantidade_de_reclamacoes_reguladas_procedentes,quantidade_de_reclamacoes_reguladas_outras,quantidade_de_reclamacoes_nao_reguladas,quantidade_total_de_reclamacoes,quantidade_total_de_clientes_ccs_e_scr,quantidade_de_cliente_ccs,quantidade_de_cliente_scr,employer_website,employer_headquarters,employer_founded,employer_industry,employer_revenue,geral,cultura_e_valores,diversidade_e_inclusao,qualidade_de_vida,alta_lideranca,remuneracao_e_beneficios,oportunidades_de_carreira,recomendam_para_outras_pessoas,perspectiva_positiva_da_empresa,cnpj,nome,segmento
2022,4o,"DEMAIS BANCOS, FINANCEIRAS E INSTITUICOES DE PAGAMENTO",BANCO/FINANCEIRA,07237373,BANCO DO NORDESTE DO BRASIL S.A.,,7,null,27,61,9384798,null,null,null,null,null,null,null,4.5,4.4,4.3,4.3,4.1,4.2,4.0,null,null,07237373,BANCO DO NORDESTE DO BRASIL S.A.,S2
2022,4o,"DEMAIS BANCOS, FINANCEIRAS E INSTITUICOES DE PAGAMENTO",BANCO/FINANCEIRA,60518222,BANCO SUMITOMO MITSUI BRASILEIRO S.A.,,0,null,1,1,1180,null,null,null,null,null,null,null,3.4,3.3,3.2,3.4,3.1,3.3,3.1,null,null,60518222,BANCO SUMITOMO MITSUI BRASIL S.A.,S3
2022,1o,"DEMAIS BANCOS, FINANCEIRAS E INSTITUICOES DE PAGAMENTO",BANCO/FINANCEIRA,58616418,BANCO FIBRA S.A.,,1,null,0,2,163906,null,null,null,null,null,null,null,4.3,4.3,4.3,3.8,3.7,4.2,3.6,null,null,58616418,BANCO FIBRA S.A.,S3
2022,3o,"DEMAIS BANCOS, FINANCEIRAS E INSTITUICOES DE PAGAMENTO",BANCO/FINANCEIRA,07237373,BANCO DO NORDESTE DO BRASIL S.A.,,9,null,48,98,9294372,null,null,null,null,null,null,null,4.5,4.4,4.3,4.3,4.1,4.2,4.0,null,null,07237373,BANCO DO NORDESTE DO BRASIL S.A.,S2
2022,3o,"DEMAIS BANCOS, FINANCEIRAS E INSTITUICOES DE PAGAMENTO",BANCO/FINANCEIRA,58616418,BANCO FIBRA S.A.,,0,null,0,1,163544,null,null,null,null,null,null,null,4.3,4.3,4.3,3.8,3.7,4.2,3.6,null,null,58616418,BANCO FIBRA S.A.,S3
2022,4o,"DEMAIS BANCOS, FINANCEIRAS E INSTITUICOES DE PAGAMENTO",BANCO/FINANCEIRA,60518222,BANCO SUMITOMO MITSUI BRASILEIRO S.A.,,0,null,1,1,1180,null,null,null,null,null,null,null,3.4,3.2,3.3,3.4,3.1,3.3,3.1,null,null,60518222,BANCO SUMITOMO MITSUI BRASIL S.A.,null
2022,1o,"DEMAIS BANCOS, FINANCEIRAS E INSTITUICOES DE PAGAMENTO",BANCO/FINANCEIRA,07237373,BANCO DO NORDESTE DO BRASIL S.A.,,9,null,31,66,9079293,null,null,null,null,null,null,null,4.5,4.4,4.3,4.3,4.1,4.2,4.0,null,null,07237373,BANCO DO NORDESTE DO BRASIL S.A.,S2
2021,2o,DEMAIS BANCOS E FINANCEIRAS,BANCO/FINANCEIRA,60518222,BANCO SUMITOMO MITSUI BRASILEIRO S.A.,,0,null,1,1,1063,null,null,null,null,null,null,null,3.4,3.3,3.2,3.4,3.1,3.3,3.1,null,null,60518222,BANCO SUMITOMO MITSUI BRASIL S.A.,S3
2021,4o,"DEMAIS BANCOS, FINANCEIRAS E INSTITUICOES DE PAGAMENTO",BANCO/FINANCEIRA,58616418,BANCO FIBRA S.A.,,1,null,0,3,164195,null,null,null,null,null,null,null,4.3,4.3,4.3,3.8,3.7,4.2,3.6,null,null,58616418,BANCO FIBRA S.A.,S3
2021,2o,DEMAIS BANCOS E FINANCEIRAS,BANCO/FINANCEIRA,60518222,BANCO SUMITOMO MITSUI BRASILEIRO S.A.,,0,null,1,1,1063,null,null,null,null,null,null,null,3.4,3.2,3.3,3.4,3.1,3.3,3.1,null,null,60518222,BANCO SUMITOMO MITSUI BRASIL S.A.,null


In [ ]:
%sql
DESCRIBE DETAIL delta.`/FileStore/default/tables/glassdoor_tb`;

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,31b8b352-f864-4f59-b95e-40d651b5724e,null,null,dbfs:/FileStore/default/tables/glassdoor_tb,2024-08-17T02:18:25.294+0000,2024-08-17T02:38:57.000+0000,List(ano),2,19721,Map(),1,2,"List(appendOnly, invariants)",Map()
